# Parametrized Sequences

In [ ]:
import numpy as np
import pulser
from pulser import Pulse, Sequence, Register
from pulser.waveforms import RampWaveform, BlackmanWaveform, CompositeWaveform
from pulser.devices import Chadoq2

From simple sweeps to variational quantum algorithms, it is often the case that one wants to try out multiple pulse sequences that vary only in a few parameters. For this effect, the ability to make a `Sequence` **parametrized** was developed.

A parametrized `Sequence` can be used just like a "regular" `Sequence`, with a few key differences. Initialization and channel declaration, for example, don't change at all: 

In [ ]:
reg = Register.square(2, prefix="q")
seq = Sequence(reg, Chadoq2)
seq.declare_channel("rydberg", "rydberg_global")
seq.declare_channel("raman", "raman_local")

## Variables and Parametrized Objects

The defining characteristic of a parametrized `Sequence` is its use of **variables**. These variables are declared within a `Sequence`, by calling:

In [ ]:
Omega_max = seq.declare_variable("Omega_max")
ts = seq.declare_variable("ts", size=2, dtype=int)
last_target = seq.declare_variable("last_target", dtype=int)

The returned `Omega_max`, `ts` and `last_target` objects are of type `Variable`, and are defined by their name, size and data type. In this case, `Omega_max` is a variable of `size=1` and `dtype=float` (the default), `ts` is an array of two `int` values and `last_target` is an `int`.

These returned `Variable` objects support simple arithmetic operations (when applicable) and, when of `size > 1`, even item indexing. Take the following examples:

In [ ]:
t_rise, t_fall = ts  # Unpacking is possible too
U = Omega_max / 2.3
delta_0 = -6 * U
delta_f = 2 * U
t_sweep = (delta_f - delta_0) / (2 * np.pi * 10) * 1000

Both the original `Variables` and the results of these operations serve as valid inputs for `Waveforms`, `Pulses` or `Sequence`-building instructions. We can take `Omega_max` as an argument for a waveform:

In [ ]:
pi_wf = BlackmanWaveform.from_max_val(Omega_max, np.pi)

or use derived quantities, like `t_rise`, `t_fall`, `delta_0` and `delta_f`:

In [ ]:
rise_wf = RampWaveform(t_rise, delta_0, delta_f)
fall_wf = RampWaveform(t_fall, delta_f, delta_0)
rise_fall_wf = CompositeWaveform(rise_wf, fall_wf)

These waveforms are *parametrized* objects, so usual attributes like `duration` or `samples` are not available, as they depend on the values of the underlying variables. Nonetheless, they can be used as regular waveforms when creating `Pulses`, which will consequently be *parametrized* too.

In [ ]:
pi_pulse = Pulse.ConstantDetuning(pi_wf, 0, 0)
rise_fall = Pulse.ConstantAmplitude(Omega_max, rise_fall_wf, 0)

## Constructing the Sequence

Upon initialization, a `Sequence` is, by default, not parametrized. We can check this by calling:

In [ ]:
seq.is_parametrized()

While it is not parametrized, it is just a normal sequence. We can do the usual stuff, like targeting a local channel, adding regular pulses, or plotting the sequence:

In [ ]:
generic_pulse = Pulse.ConstantPulse(100, 2 * np.pi, 2, 0.0)
seq.add(generic_pulse, "rydberg")
seq.target("q0", "raman")
seq.add(generic_pulse, "raman")
seq.draw()

The `Sequence` becomes parametrized at the moment a parametrized object or variable is given to a sequence-building instruction. For example:

In [ ]:
seq.target_index(last_target, "raman")
seq.is_parametrized()

From this point onward, functionalities like drawing are no longer available, because the instructions start being stored instead of executed on the fly. We can still check the current state of a parametrized sequence by printing it:

In [ ]:
print(seq)

Naturally, we can also add the parametrized pulses we previously created:

In [ ]:
seq.add(rise_fall, "rydberg")
seq.add(pi_pulse, "raman")

## Building

Once we're happy with our parametrized sequence, the last step is to build it into a regular sequence. For that, we call the `Sequence.build()` method, in which we **must attribute values for all the declared variables**:

In [ ]:
built_seq = seq.build(
    Omega_max=2.3 * 2 * np.pi, ts=[200, 500], last_target=reg.find_indices(["q3"])[0]
)
built_seq.draw()

And here we have a regular sequence, built from our parametrized sequence. To create a new one with different parameters, we can simply build it again with new values:

In [ ]:
alt_seq = seq.build(Omega_max=2 * np.pi, ts=[400, 100], last_target=reg.find_indices(["q2"])[0]
alt_seq.draw()